In [35]:
import numpy as np
from itertools import combinations

def find_common_variable_nodes(H, syndrome):
    """
    Find common variable nodes connected to unsatisfied check nodes.
    Note: When there is only one unsatisfied check node, then it finds the variable node that is connected to it but not to any satisfied check node.
    When all check nodes are satisfied, then we obtain an empty dictionary.

    Parameters:
        H (numpy.ndarray): Parity-check matrix (m x n).
        syndrome (numpy.ndarray): Syndrome vector (1 x m).

    Returns:
        dict: A dictionary where keys are variable node indices and values are their counts.
    """
    ## Invalid input dimensions
    m, n = H.shape
    if len(syndrome) != m:
        raise ValueError("Dimensions of H and syndrome do not match.")

    ## To find indices where syndrome is non-zero (unsatisfied check nodes)
    unsatisfied_check_indices = np.where(syndrome != 0)[0]
    num_unsatisfied = len(unsatisfied_check_indices)

    ## To store variable nodes and their counts
    variable_counts = {}

    ## Case where there are no unsatisfied check nodes
    if num_unsatisfied == 0:
        return variable_counts

    ## Case where there is only one unsatisfied check node
    if num_unsatisfied == 1:

        ## Unsatisfied check node
        unsatisfied_check_index = unsatisfied_check_indices[0]

        ## Variable nodes connected to the unsatisfied check node
        unsatisfied_variable_nodes = set(np.where(H[unsatisfied_check_index, :] == 1)[0])

        ## Satisfied check nodes
        satisfied_check_indices = np.where(syndrome == 0)[0]

        ## Variable nodes connected to the satisfied check nodes
        satisfied_variable_nodes = set()
        for check_index in satisfied_check_indices:
            satisfied_variable_nodes.update(np.where(H[check_index, :] == 1)[0])

        ## Variable nodes that are connected to the unsatisfied check node but not to any satisfied check node
        uncommon_variable_nodes = unsatisfied_variable_nodes - satisfied_variable_nodes

        for var_node in uncommon_variable_nodes:
            variable_counts[var_node] = 1

        return variable_counts

    ## To find common variable nodes in all possible combinations of unsatisfied check nodes
    if num_unsatisfied > 1:
        ## To keep track of processed combinations of check nodes
        processed_combinations = set()

        ## Begin with the largest combination (of all unsatisfied check nodes) and work down to pairs
        for r in range(num_unsatisfied, 1, -1):
            ## Generate all combinations of size r
            for combo in combinations(unsatisfied_check_indices, r):
                ## Skip if this combination is a subset of any previously processed combination
                if any(set(combo).issubset(processed) for processed in processed_combinations):
                    continue

                ## To find the variable nodes for the current combination of check nodes
                variable_node_sets = []
                for check_index in combo:
                    row = H[check_index, :]
                    variable_nodes = np.where(row == 1)[0]
                    variable_node_sets.append(set(variable_nodes))

                ## Common variable nodes for a combination
                common_nodes = set.intersection(*variable_node_sets)

                if common_nodes:
                    for var_node in common_nodes:
                        if var_node in variable_counts:
                            variable_counts[var_node] += len(combo)
                        else:
                            variable_counts[var_node] = len(combo)

                    ## Processed combination
                    processed_combinations.add(frozenset(combo))

    return variable_counts

def flip_variable_nodes(H, syndrome, received_codeword):
    """
    Flip variable nodes in the received codeword based on the threshold (number of ones in each column of H).
    Note: It returns the original codeword only if there is one bit error in the received codeword.

    Parameters:
        H (numpy.ndarray): Parity-check matrix (m x n).
        syndrome (numpy.ndarray): Syndrome vector (1 x m).
        received_codeword (numpy.ndarray): Received codeword (1 x n).

    Returns:
        numpy.ndarray: Decoded codeword (1 x n).
    """
    ## Find common variable nodes and their counts
    variable_counts = find_common_variable_nodes(H, syndrome)

    ## Compute the threshold for each variable node (i.e, number of ones in each column of H)
    thresholds = np.sum(H, axis=0)

    decoded_codeword = received_codeword.copy()

    ## Flip variable nodes where the count exceeds or equals the threshold
    for var_node, count in variable_counts.items():
        if count >= thresholds[var_node]:
            decoded_codeword[var_node] = 1 - decoded_codeword[var_node]

    return decoded_codeword


# # Define a parity-check matrix H
# H = np.array([
#     [1, 1, 0, 1, 0, 0],
#     [0, 1, 1, 0, 1, 0],
#     [1, 1, 1, 0, 0, 1]
# ])

# H = np.array([
#     [1, 1, 0, 1, 0, 0],
#     [0, 1, 1, 0, 1, 0],
#     [1, 0, 0, 0, 1, 1],
#     [0, 0, 1, 1, 0, 1]
# ])

original_codeword = np.array([0, 0, 1, 0, 1, 1])
error_pattern = np.array([0, 0, 0, 0, 0, 1])

received_codeword = (original_codeword + error_pattern) % 2

print("Original Codeword:", original_codeword)
print("Error Pattern:", error_pattern)
print("Received Codeword:", received_codeword)

syndrome = np.mod(H @ received_codeword, 2)

print("Syndrome:", syndrome)

decoded_codeword = flip_variable_nodes(H, syndrome, received_codeword)

print("Decoded Codeword:")
print(decoded_codeword)

Original Codeword: [0 0 1 0 1 1]
Error Pattern: [0 0 0 0 0 1]
Received Codeword: [0 0 1 0 1 0]
Syndrome: [0 0 1]
Decoded Codeword:
[0 0 1 0 1 1]
